# Streamlit Rule Builder
Interactive editor + preview + approve.

In [ ]:
import streamlit as st
from pyspark.sql import functions as F

st.title("Risk Rules – Builder & Approval")

# Inputs
catalog = st.text_input("Catalog", value="<CATALOG_NAME>")
st.caption("Set at Job runtime or here for ad-hoc runs.")
_ = spark.sql(f"USE CATALOG {catalog}")
spark.sql("USE SCHEMA control")

# Load rules
rules = spark.table("control.risk_rules")
st.subheader("Rules")
st.dataframe(rules.toPandas())

# Simple toggle UI (edit enable/priority inline)
with st.expander("Enable / Disable rules"):
    rule_ids = [r["rule_id"] for r in rules.select("rule_id").collect()]
    selected = st.multiselect("Enable only these rules (others disabled):", rule_ids, rule_ids)
    if st.button("Apply enable/disable"):
        spark.sql("UPDATE control.risk_rules SET enabled = false")
        if selected:
            spark.sql(f"UPDATE control.risk_rules SET enabled = true WHERE rule_id IN ({','.join([f"'{x}'" for x in selected])})")
        st.success("Rule enablement updated.")

st.subheader("Preview impact (1% sample)")
if st.button("Run Preview"):
    spark.sql("USE SCHEMA gold")
    f = spark.table("gold.features").sample(0.01, seed=42).alias("f")
    r = spark.table("control.risk_rules").where("enabled = true AND current_date BETWEEN effective_from AND coalesce(effective_to, date'2999-12-31')")
    matches = None
    for row in r.collect():
        pred = row["condition_sql"]
        part = f.selectExpr(
            "loan_id",
            f"'{row['rule_id']}' as rule_id",
            f"'{row['impact_column']}' as impact_column",
            f"'{row['impact_value']}' as impact_value",
            f"{row['priority']} as priority"
        ).where(pred)
        matches = part if matches is None else matches.unionByName(part)
    if matches is None:
        preview = f.withColumn("matched_rules", F.array())\
                   .withColumn("risk_band", F.lit("Medium"))\
                   .withColumn("risk_points", F.lit(0))
    else:
        from pyspark.sql.window import Window
        w = Window.partitionBy("loan_id","impact_column").orderBy(F.desc("priority"))
        top = matches.withColumn("rn", F.row_number().over(w)).where("rn=1")
        preview = (f.join(top, on="loan_id", how="left")
                     .groupBy(f.columns)
                     .agg(F.collect_list("rule_id").alias("matched_rules"),
                          F.max(F.when(F.col("impact_column")=="risk_band", F.col("impact_value"))).alias("risk_band"),
                          F.sum(F.when(F.col("impact_column")=="risk_points",
                                       F.regexp_extract(F.col("impact_value"), "[-+]?\d+", 0).cast("int")).otherwise(0)).alias("risk_points")))\
                     .fillna({"risk_band":"Medium","risk_points":0,"matched_rules":[]})
    st.write("Sample KPIs:")
    st.write(preview.select(F.count("*").alias("loans_total")).toPandas())
    st.write(preview.select(F.avg("dti").alias("avg_dti"), F.avg("fico_score").alias("avg_fico")).toPandas())
    st.write(preview.select(F.sum((F.col("risk_band")=="High").cast("int")).alias("high_risk_count")).toPandas())
    st.success("Preview complete.")

st.subheader("Generate Draft Report")
if st.button("Build Draft Now"):
    spark.sql("USE SCHEMA gold")
    from datetime import datetime
    run_id = f"RR_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"
    spark.sql(f"INSERT INTO gold.report_runs VALUES ('{run_id}','Risk Report','DRAFT','rules@current',current_timestamp(),NULL,NULL,NULL)")
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW _kpi AS
        SELECT 'loans_total' as metric, 'all' as dimension, count(*)*1.0 as value FROM gold.risk_eval
        UNION ALL
        SELECT 'high_risk_count','all', sum(CASE WHEN risk_band='High' OR risk_points>=20 THEN 1 ELSE 0 END) FROM gold.risk_eval
        UNION ALL
        SELECT 'avg_dti','all', avg(dti) FROM gold.risk_eval
        UNION ALL
        SELECT 'avg_fico','all', avg(fico_score) FROM gold.risk_eval
    """)
    spark.sql(f"DELETE FROM gold.report_facts WHERE report_run_id = '{run_id}'")
    spark.sql(f"INSERT INTO gold.report_facts SELECT '{run_id}', metric, dimension, value FROM _kpi")
    st.success(f"Draft report created: {run_id}")

st.subheader("Approve Latest Draft")
if st.button("Approve Most Recent Draft"):
    latest = spark.sql("SELECT report_run_id FROM gold.report_runs WHERE status='DRAFT' ORDER BY started_at DESC LIMIT 1").collect()
    if latest:
        rid = latest[0]["report_run_id"]
        spark.sql(f"""
            UPDATE gold.report_runs
            SET status='APPROVED', approved_by=current_user(), approved_at=current_timestamp()
            WHERE report_run_id='{rid}' AND status='DRAFT'
        """)
        spark.sql("""
            CREATE OR REPLACE VIEW gold.report_facts_approved_latest AS
            SELECT rf.*
            FROM gold.report_facts rf
            JOIN (
              SELECT report_run_id FROM gold.report_runs WHERE status='APPROVED' ORDER BY approved_at DESC LIMIT 1
            ) latest USING (report_run_id);
        """)
        st.success(f"Approved run: {rid}")
    else:
        st.warning("No draft runs found.")
